# 多任务编程——进程

## 一、多任务介绍

现有的知识下只能单任务的执行程序，多任务就是为了充分利用CPU资源，提高程序运行效率。

如今的操作系统都是多任务的，如同时运行Powershell、Edge、Tim等

### 并发

一段时间交替执行任务。最早的CPU单核，执行多任务只能是让每个任务执行一小段时间（为每个任务分配时间线），多个任务交替执行。

任务数＞计算机核数

### 并行

（多核CPU）真正的同时执行多个任务，同一时间内一起运行（与交替运行区分）。

多核CPU才能让多个软件真正的一起运行。

任务数$\le$核数

当任务数大于核数，又会为每个任务分配时间线，执行并发的多程序运行。

## 二、进程

进程：一个**正在运行**的程序或者软件。

启动进程，操作系统要为进程分配对应的（内存）资源，真正运行程序（干活的实体）的是线程。

注意：

一个程序至少有一个一个进程，一个进程默认有一个线程（可以创建多个线程）。线程依附线程而存在，没有进程就没有线程。

程序若创建多个进程，多个进程彼此独立的工作。

**进程是系统分配资源的基本单位（操作系统分配资源就产生进程），进程本身不执行任务，只负责向操作系统索要资源，真正执行代码的是线程**

## 三、多进程的使用

```py
import multiprocessing

multiprocessing.Process(group[, target[, name[, args[, kwargs]]]])  # 进程类
```

group:指定进行组，目前只能使用None

target：执行的任务名（重要，为进程指定任务：一个函数或一个方法）

name：进程名字（默认，从1开始产生Process-1）

args：元组方式传参

kwargs：字典方式传参

Process类的常用方法：

start() 启动子进程实例

join() 等待子进程结束

terminate() 不管任务是否完成，立即终止子进程

Process类的常用属性：

name 获取进程名

程序启动会产生主进程，可以在主进程中手动创建子进程。

In [ ]:
import multiprocessing
import time


def sing():
    for i in range(3):
        print("Sing a song!")
        time.sleep(1)


def dance():
    for i in range(3):
        print("Dance")
        time.sleep(1)


if __name__ == '__main__':  # 缺少会报错
    process = multiprocessing.Process(target=sing)

    process.start()  # 子进程

    dance()  # 在主进程中执行
# Dance
# Sing a song!
# Dance
# Sing a song!
# Dance
# Sing a song!

进程之间的执行是无序的，取决于系统的调度。

In [ ]:
import multiprocessing
import time


def sing():
    for i in range(3):
        print("Sing a song!")
        time.sleep(1)


def dance():
    for i in range(3):
        print("Dance")
        time.sleep(3)


if __name__ == '__main__':  # 缺少会报错
    process1 = multiprocessing.Process(target=sing)
    process2 = multiprocessing.Process(target=dance)

    process1.start()
    process2.start()


以上就没有在主进程中执行代码，而是创建了两个子进程。

## 四、获取进程编号

获取进程编号的目的是验证主进程和子进程的关系，可以得知子进程是由哪个主进程创建出来的。

两种操作：

获取当前进程编号  os.getpid() -> int

获取当前进程父进程编号  os.getppid() -> int

mutiprocessing.current_process()可以获得当前进程。

In [ ]:
import multiprocessing
import time
import os


def sing():
    print("父进程编号为", os.getppid())
    print("进程编号为", os.getpid())
    for i in range(3):
        print("Sing a song!")
        time.sleep(1)


def dance():
    for i in range(3):
        print("Dance")
        time.sleep(3)


if __name__ == '__main__':  # 缺少会报错
    print(multiprocessing.current_process())  # <_MainProcess name='MainProcess' parent=None started>
    # MainProcess代表主进程
    
    process1 = multiprocessing.Process(target=sing)  # <Process name='Process-1' parent=12036 initial>
    print(process1)
    process2 = multiprocessing.Process(target=dance, name="dance") # <Process name='dance' parent=12036 initial>
    print(process2)  # 显示成了设置的名字

In [ ]:
import multiprocessing
import time
import os


def sing():
    print("进程编号为", os.getpid())
    print("父进程编号", os.getppid())
    for i in range(3):
        print("Sing a song!")
        time.sleep(1)


def dance():
    print("进程编号为", os.getpid())
    print("父进程编号", os.getppid())
    for i in range(3):
        print("Dance")
        time.sleep(3)


if __name__ == '__main__':  # 缺少会报错
    print(multiprocessing.current_process())  # <_MainProcess name='MainProcess' parent=None started>
    print("主进程编号", os.getpid())
    process1 = multiprocessing.Process(target=sing)  # <Process name='Process-1' parent=12036 initial>
    print(process1)
    process2 = multiprocessing.Process(target=dance, name="dance") # <Process name='dance' parent=12036 initial>
    print(process2)

    process1.start()
    process2.start()

类似linux **kill**命令，使用os.kill()可以在程序中杀死进程。

In [ ]:
def sing():
    print("进程编号为", os.getpid())
    print("父进程编号", os.getppid())
    for i in range(3):
        print("Sing a song!")
        os.kill(os.getpid(), 9)
        time.sleep(1)

只会输出一次Sing a song，参数9的含义表示强制杀死这个进程。

## 五、进程执行带参数的任务

传参args关键字以元组形式传参，kwargs以字典形式传参。

In [ ]:
import multiprocessing


def show_info(name, age, gender):
    print(f'name={name}, age={age}, gender={gender}')


if __name__ == '__main__':
    # 执行的是函数和方法，本质就都是函数
    process1 = multiprocessing.Process(target=show_info, args=('wuhaoze', 20, 'male'))
    process1.start()

    process2 = multiprocessing.Process(target=show_info, kwargs={'age': 20, 'name': 'wuhaoze', 'gender': 'male'})
    process2.start()

    process3 = multiprocessing.Process(target=show_info, args=('wuhaoze',), kwargs={'age': 20, 'gender': 'male'})
    process3.start()
    # 类似于函数参数传递，前面的参数必须按照位置，后面的使用关键字


创建进程指定target不能加括号，否则就是直接调用函数。

## 六、多进程需要注意的特性

多进程任务的注意点：

1.进程之间不共享全局变量

2.主进程会等待所有子进程结束再结束

### 1.进程之间不共享全局变量

In [1]:
import multiprocessing
from time import sleep


g_list = []


def add_data():
    for i in range(3):
        g_list.append(i)  # 可变类型的修改不需要global关键字声明
        print(g_list)
        sleep(0.6)


def read_data():
    print(g_list)  # []


if __name__ == '__main__':
    process1 = multiprocessing.Process(target=add_data)
    process1.start()
    # 当前进程（主进程）等待process1执行完，再向下执行
    process1.join()

    process2 = multiprocessing.Process(target=read_data)
    process2.start()

    print(g_list)  # []


[]


创建子进程其实是对主进程资源的拷贝（主进程有什么代码，子进程就有什么代码），子进程就是主进程的一个副本。副本的修改不会影响其他进程。

In [ ]:
import multiprocessing
from time import sleep


g_list = [0]


def add_data():
    for i in range(3):
        g_list.append(i)  # 可变类型的修改不需要global关键字声明
        print(g_list)
        sleep(0.6)


def read_data():
    print(g_list)  # []


if __name__ == '__main__':  # Unix中代码不会进行拷贝执行，windows会，所以必须对名称做条件判定
    process1 = multiprocessing.Process(target=add_data)
    process1.start()
    # 当前进程（主进程）等待process1执行完，再向下执行
    process1.join()

    g_list.append(100)

    process2 = multiprocessing.Process(target=read_data)
    process2.start()
    process2.join()

    print(g_list)


process2的输出仍是空列表，因为实质拷贝的是代码，g_list = []进行了拷贝，但对于列表的修改拷贝了却无法执行。只拷贝判断主模块上方的代码。

加上判断主模块的条件也更有全局变量的感觉。

\_\_mian\_\_也有了第二个意义，windows下避免多进程创建时递归拷贝代码创建子进程。

### 2.主进程会等待所有子进程执行结束再结束

In [ ]:
import multiprocessing
from time import sleep


def task():
    for i in range(10):
        print('haha')


if __name__ == '__main__':
    sub_process = multiprocessing.Process(target=task)

    sub_process.start()

    print('over')


希望主进程结束时，不等待其他进程

方法1：子进程守护主进程

In [ ]:
import multiprocessing
from time import sleep


def task():
    for i in range(100):
        print('haha')


if __name__ == '__main__':
    sub_process = multiprocessing.Process(target=task)

    sub_process.daemon = True  # 守护主进程
    sub_process.start()

    sleep(3)
    print('over')

方法2：主进程结束之前终止子进程（无论是否任务执行完都终止）

In [ ]:
import multiprocessing
from time import sleep


def task():
    for i in range(100):
        print('haha')


if __name__ == '__main__':
    sub_process = multiprocessing.Process(target=task)

    sub_process.start()

    sleep(3)
    sub_process.terminate()
    print('over')